Задание 1

| Описание                        | Архитектура  | Объяснение                                                               |
|---------------------------------|--------------|--------------------------------------------------------------------------|
| Классификация «позитив/негатив» | Encoder-only | Требуется только понимание входного текста без генерации нового текста   |
| Продолжение текста              | Decoder-only | Авторегрессионная генерация следующего токена на основе предыдущих       |
| Машинный перевод                | Encoder-only | Преобразование входного текста на одном языке в выходной текст на другом |
| Заполнение пропущенного слова   | Encoder-only | Анализ контекста для предсказания пропущенного элемента                  |
| Ответ в диалоге                 | Decoder-only | Генерация ответа на основе предыдущего контекста диалога                 |
| Кодирование документа           | Encoder-only | Создание векторного представления документа для дальнейшей обработки     |

Задание 2

In [ ]:
distribution = {
    "flowers": 0.61,
    "plants": 0.25,
    "trees": 0.07,
    "animals": 0.02,
    "garden": 0.05
}

greedy_token = max(distribution, key=distribution.get)
print(f"'{greedy_token}' (вероятность: {distribution[greedy_token]})")

'flowers' (вероятность: 0.61)


In [5]:
import numpy as np

def manual_sampling(prob_dict):
    tokens = list(prob_dict.keys())
    probs = list(prob_dict.values())
    cumulative_probs = np.cumsum(probs)
    rand_val = np.random.random()
    
    for i, cum_prob in enumerate(cumulative_probs):
        if rand_val <= cum_prob:
            return tokens[i]
    return tokens[-1]

sampled_token = manual_sampling(distribution)
print(f"'{sampled_token}'")

'plants'


Модель использует семантические связи между словами. У flowers и plants вероятность высокая, поскольку они оба относятся к тематике растений вида цветов. У animals вероятность низкая, поскольку эта тема менее релевантна в текущем контексте. Модель показывает статистические закономерности языка и ассоциации между понятиями.

Задание 3

In [18]:
logits = {
    "flowers": 2.1,
    "plants": 1.8,
    "trees": 0.5,
    "animals": -0.4
}

def softmax_with_temperature(logits_dict, tau=1.0):
    # Извлечь значения логотипов 
    values = np.array(list(logits_dict.values()))
    
    # Применить т и softmax
    scaled_logits = values / tau
    exp_values = np.exp(scaled_logits - np.max(scaled_logits))  # Стабильная версия
    probabilities = exp_values / np.sum(exp_values)
    
    # Вернуть словарь с результатами
    return dict(zip(logits_dict.keys(), probabilities))

def print_probabilities(probs): # Для нормального вывода
    for category, prob in probs.items():
        print(f"  {category}: {prob:.3f}")

1. При τ = 1

In [19]:
probs_tau1 = softmax_with_temperature(logits, tau=1.0)
print_probabilities(probs_tau1)

  flowers: 0.494
  plants: 0.366
  trees: 0.100
  animals: 0.041


In [21]:
probs_tau05 = softmax_with_temperature(logits, tau=0.5)
print_probabilities(probs_tau05)

  flowers: 0.626
  plants: 0.344
  trees: 0.026
  animals: 0.004


In [22]:
probs_tau2 = softmax_with_temperature(logits, tau=2.0)
print_probabilities(probs_tau2)

  flowers: 0.385
  plants: 0.331
  trees: 0.173
  animals: 0.110


2. При τ = 0.5 (<1) распределение "пиковое", доминирование вариантов становится сильнее, разнообразие меньше. При τ = 2.0 (>1) распределение более гладкое, вероятности выравниваются, разнообразие выборки больше

3. Модель будет вести себя почти как greedy, когда τ будет стримиться к 0, чтобы распределение вероятностей было максимально "пиковым", так как при greedy модель всегда выбирает вариант с наибольшей вероятностью (самый вероятный класс получает вероятность 1, а все остальные — 0.)